# Generates crops ad pickle file using dataset_gen_pipe

This notebook crops images according to bounding box coordinates (one per sulcus)

# Imports

In [41]:
import sys
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the aims module
from soma import aims
# the brainplot package
import colorado as cld

print((sys.version))

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


In [42]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)

{'xvfb': <subprocess.Popen object at 0x7ff55052b128>, 'original_display': ':1', 'display': 2, 'glx': 2, 'virtualgl': None, 'headless': True, 'mesa': False, 'qtapp': None}


The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [43]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print((os.path.dirname(deep_folding.__file__)))

/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/deep_folding


In [44]:
_ALL_SUBJECTS = -1

# User-specific variables

In [45]:
sulcus = 'S.T.s.ter.asc.ant._left'

In [46]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [47]:
src_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

src_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/source/unsupervised


In [48]:
tgt_dir = os.path.join(os.getcwd(), '../data/target')
tgt_dir = os.path.abspath(tgt_dir)
print(("tgt_dir = " + tgt_dir))

tgt_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target


The target directory tgt_dir is where the files will be saved

In [49]:
transform_dir = os.path.join(os.getcwd(), '../data/reference/transform')
transform_dir = os.path.abspath(transform_dir)
print(("transform_dir = " + transform_dir))

transform_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/reference/transform


In [50]:
bbox_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
bbox_dir = os.path.abspath(bbox_dir)
print(("bbox_dir = " + bbox_dir))

bbox_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/reference/bbox


In [51]:
mask_dir = os.path.join(os.getcwd(), '../data/target/mask')
mask_dir = os.path.abspath(mask_dir)
print(("mask_dir = " + mask_dir))

mask_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/mask


In [52]:
print((sys.argv))

['anatomist']


# Illustration of main program uses

We will first use the program with no effect by using number of subjects set to 0, or by calling the help function

### Using external calls

In [53]:
!python ../deep_folding/anatomist_tools/dataset_gen_pipe.py -n 0 -t tgt_local_dir

In [54]:
# Clean
!rm -rf tgt_local_dir

In [55]:
!python ../deep_folding/anatomist_tools/bounding_box.py --help

python: can't open file '../deep_folding/anatomist_tools/bounding_box.py': [Errno 2] No such file or directory


### By using the main function call

In [56]:
from deep_folding.anatomist_tools import dataset_gen_pipe
print((dataset_gen_pipe.__file__))

/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/deep_folding/anatomist_tools/dataset_gen_pipe.py


In [57]:
args = "-n 0 -t " + tgt_dir
argv = args.split(' ')

In [58]:
dataset_gen_pipe.main(argv)

In [59]:
args = "--help"
argv = args.split(' ')

In [60]:
dataset_gen_pipe.main(argv)

usage: dataset_gen_pipe.py [-h] [-s SRC_DIR] [-t TGT_DIR] [-a MASK_DIR]
                           [-b BBOX_DIR] [-m MORPHOLOGIST_DIR]
                           [-u SULCUS [SULCUS ...]] [-i SIDE] [-n NB_SUBJECTS]
                           [-e INTERP] [-p RESAMPLING] [-c CROPPING]
                           [-v OUT_VOXEL_SIZE [OUT_VOXEL_SIZE ...]]

Generates cropped and pickle files

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR, --src_dir SRC_DIR
                        Source directory where the MRI data lies. Default is :
                        /neurospin/hcp
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the cropped and pickle
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test
  -a MASK_DIR, --mask_dir MASK_DIR
                        masking directory where mask has been stored. Default
                        is : /neurospin/dico/data/deep_

### By using the API function call

In [61]:
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir,
                                  bbox_dir=bbox_dir,
                                  cropping='bbox',
                                  list_sulci=sulcus,
                                  side=side,
                                  number_subjects=0)

In [62]:
def are_arrays_almost_equal(arr1, arr2, epsilon, max_number_different_pixels):
    """Returns True if at most max_number_different_pixels pixels of arrays arr1 and arr2 
    differ by more than epsilon
    
    """
    difference = (abs(arr1-arr2) >= epsilon)
    number_different_pixels = np.count_nonzero(difference)
    return number_different_pixels <= max_number_different_pixels, number_different_pixels

# Crops with mask and with nearest-neighbour interpolation

## Main program

In [63]:
tgt_dir_nearest = os.path.join(os.getcwd(), '../data/target/crops/nearest')
tgt_dir_nearest = os.path.abspath(tgt_dir_nearest)
print(("tgt_dir = " + tgt_dir_nearest))

tgt_dir = /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/crops/nearest


In [64]:
src_dir

'/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/source/unsupervised'

In [65]:
interp = 'nearest'
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir_nearest,
                                  mask_dir=mask_dir,
                                  cropping="mask",
                                  list_sulci=sulcus,
                                  side=side,
                                  interp=interp,
                                  number_subjects=1)

['100206']


## Result analysis

### Analysis of the inputs

In [66]:
# Gets source file as numpy array
skeleton_dir = os.path.join(src_dir, "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation")
vol_source_file = glob.glob(skeleton_dir + '/' + side + '*.nii.gz')[0]
vol_source = aims.read(vol_source_file)
arr_source = vol_source.arraydata()
print("shape of source skeleton = ", arr_source.shape)

shape of source skeleton =  (1, 260, 311, 260)


In [67]:
np.unique(arr_source)

array([ 0, 10, 11, 30, 40, 60, 80], dtype=int16)

In [68]:
pd.value_counts(np.resize(arr_source, arr_source.size))

11    18832738
0      2012736
60      163069
30        9634
80        5330
40          87
10           6
dtype: int64

In [69]:
mask_file = glob.glob(mask_dir + '/' + side + '/*.nii.gz')[0]
vol_mask = aims.read(mask_file)
arr_mask = vol_mask.arraydata()
print("shape of mask = ", arr_mask.shape)

shape of mask =  (1, 193, 229, 193)


In [70]:
np.unique(arr_mask)

array([0, 1], dtype=int16)

### Analysis of the outputs

Prints the list of files of the target directory

In [71]:
print("Files in crops target directory:")
print(tgt_dir_nearest)
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops target directory:
/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/crops/nearest
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [72]:
tgt_json_file = glob.glob(tgt_dir_nearest + '/*.json')[0]
print("tgt_json_file = ", tgt_json_file, '\n')
with open(os.path.join(tgt_dir_nearest, tgt_json_file), 'r') as f:
    data_tgt = json.load(f)
    print((json.dumps(data_tgt, sort_keys=True, indent=4)))

tgt_json_file =  /volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/crops/nearest/Ldataset.json 

{
    "bbmax": [
        157,
        171,
        110,
        1
    ],
    "bbmin": [
        129,
        150,
        60,
        0
    ],
    "bbox_dir": "/neurospin/dico/data/deep_folding/data/bbox",
    "cropped_dir": "/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/crops/nearest/Lcrops",
    "date": "2021-10-22 15:45:36",
    "git_sha": "6b623309d2d11d4832bb2f47421dcf5f0485c986",
    "interp": "nearest",
    "is_git": true,
    "list_sulci": [
        "S.T.s.ter.asc.ant._left"
    ],
    "mask_dir": "/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/mask",
    "nb_subjects": 1,
    "out_voxel_size": [
        1,
        1,
        1
    ],
    "repo_working_dir": "/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding",
    "resampling_type": "AimsApplyTransform",
    "side": "L",
    "src_dir": "/v

Obtained output (we read the cropped file from the target directory):

In [73]:
# Gets target crop as numpy array
cropped_target_dir = os.path.join(tgt_dir_nearest, side+'crops')
vol_target_file = glob.glob(cropped_target_dir + '/' + '*.nii.gz')
vol_target = aims.read(vol_target_file[0])
arr_target = vol_target.arraydata()
print("shape of target cropped image = ", arr_target.shape)

shape of target cropped image =  (1, 50, 21, 28)


In [74]:
np.unique(arr_target)

array([ 0, 11, 30, 60, 80], dtype=int16)

The scope here is to compare the different numbers present on the target array and on the source array:

In [75]:
np.around(pd.value_counts(np.resize(arr_target, arr_target.size))/arr_target.size*100, 1)

0     98.9
60     0.6
11     0.3
30     0.1
80     0.0
dtype: float64

In [76]:
np.around(pd.value_counts(np.resize(arr_source, arr_source.size))/arr_source.size*100, 1)

11    89.6
0      9.6
60     0.8
30     0.0
80     0.0
40     0.0
10     0.0
dtype: float64

### Visualization

In [77]:
print("Files in crops nearest target directory:")
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops nearest target directory:
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [78]:
target_file_dir = tgt_dir_nearest + '/' + side + 'crops'
target_file_nearest = glob.glob(target_file_dir + "/*.nii.gz")[0]
print(target_file_nearest)

/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/target/crops/nearest/Lcrops/100206_normalized.nii.gz


In [79]:
# load source skeleton data (the SliceableObject)
object_anat = a.loadObject(target_file_nearest)

# create an Axial window in anatomist
w = a.createWindow('Axial')
w.addObjects(object_anat)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [80]:
print(vol_source_file)
# load source skeleton data (the SliceableObject)
a_source = a.loadObject(vol_source_file)
print(a_source)
# create an Axial window in anatomist
w1 = a.createWindow("Axial")
w1.addObjects(a_source)

/volatile/jc225751/Runs/19_generate_crops/Program/deep_folding/data/source/unsupervised/ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation/Lskeleton_100206.nii.gz


AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)